# Creating subcategory_dictionary 
# [TODO] - extract out the page logics while deleting other create dictionary files
- same them into csv files.

In [4]:
import numpy as np
import pandas as pd
import re
import math
import unicodedata

In [5]:
# file paths
category_file = "raw_data/商品分類表_180410.xlsx"

# for category data
category_df = pd.read_excel(category_file,  sheet_name='Sheet1', index_col=0 )
category_df.columns = ['product_name',
                       'material_type_id',
                       'material_type',
                       'manufacture_country_id',
                       'manufacture_country',
                       'category_id',
                       'category']
category_df.index.name = 'product_id'

category_df

,product_name,material_type_id,material_type,manufacture_country_id,manufacture_country,category_id,category
product_id,,,,,,,
AAH080-C,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-E,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-H,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-J,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-K,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-M,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-N,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-R,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-S,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー


In [18]:
query_keywords = [
    'グラデーションスカート',
    'スカート',
    'ウッドプレート',
    'ネックレス',
    'ｈａｔ',
    'ハット',
    'キャニスター',
    'テラコッタ',
    'スリッポン',
    'スタンドミラー',
    'ピアス',
    'バッグ',
    'ｂａｇ',
    'スリッパ',
    'グローブ',
    'キャップ',
    'ｃａｐ',
    '携帯ストラップ',
    'ベッドスプレッド',
    'ハンカチ',
    'リサイクル缶',
    'ランジェ',
    'バスケット',
    'ベルト',
    'ボトルネック',
    'スヌード',
    'カーテン',
    'カッティングボード',
    'キャンドル',
    'ポーチ',
    'バケツ',
    'マフラー',
    'ケープショール',
    'ロザリオ',
    'ピッチャー',
    'ウェッジサンダル',
    'ディッシュクロス',
    'ヘアーバンド',
    'ショートパンツ',
    'マット',
    'テーブルクロス',
    'ソックス',
    'ワンピース',
    'アームウォーマー',
    'ドロワー',
    'ヘアークリップ',
    'バレエシューズ',
    'ミラー',
    'フレーム',
    'ストール',
    'ピンクッション',
    'ホルダー',
    'ミトン',
    'ヘアーピン',
    'ｊａｒ',
    'コースター',
    'ケース',
    'ブレスレット',
    'バーハンガー',
    'スカーフピン',
    'レッグウォーマー',
    'ショートポンチョ',
    'ヴェール',
    'ブーツルームシューズ',
    'チキンウィーブ',
    'クリーパーランプ',
    'ランプシェード',
    'キャニスター',
    'セーター',
    'サーブスプーン',
    'キッチンパッド',
    'ブラウス',
    'ワードローブ',
    'バレッタ',
    'テーブルセンター',
    'チュニック',
    'プレート',
    'パフューム',
    'ルームシューズ',
    'コンチャ',
    'リング',
    'カードスタンド',
    'ディスプレイカバー',
    'クッションカバー',
    'ガーデンポット',
    'ニットピン',
    'フルエプロン',
    'ディンオープントゥシューズ',
    'バングル',
    'ブルクロス',
    'プランプイアー',
    'フック',
    'オーナメント',
    'リュースガラス',
    'リューズガラス',
    'カーディガン',
    'タンクトップ',
    '３連ブレス',
    'ストラップ',
    'フラワーボウル',
    '腕時計',
    'クルタシャツ',
    'デッキシューズ',
    'フォーク',
    'ドアノブ',
    'ソープディッシュ',
    'フラックス',
    'ヘアーゴム',
    '和しらべスプーン',
    'エプロン',
    'スタンド',
    'ショート',
    'ウォレット',
    'スリープピン',
    'タブレットトレイ',
    'キッチンラック',
    'ウォーターポット',
    'ヨガドレス',
    'ホールドピン',
    'フォローラインパンツ',
    'モンブランヘアゴム',
    'シュシュ',
    'マグネット',
    'ｋｅｙチャーム',
    'ディスプレイケージ',
    'ヘアゴム',
    'キッチンクロス',
    '時計',
    'ツウィルシューズ',
    'サンシェード',
    'ソープデッシュ',
    'デイリーウェア',
    'レギンス',
    'キルトクッション',
    'スコップ',
    'レターｂｏｘ',
    'フラットトレイ',
    'ハンガーラック',
    'スニーカー',
    'カップ',
    'チェア',
    'ｕｓａステッカー',
    'ソートトレー',
    'ウォール ハンギング',
    'ペンダントコード',
    'ポ−チ',
    'ワイヤーリボン',
    'ピローカバー',
    'キャミソール',
    '封筒',
    'ブラシ',
    'ヘアバンド',
    'うちわ',
    'ディナースプーン',
    'トレーｂｏｘ',
    'クリップ',
    'ピック',
    'ベル',
    'トレー',
    '手袋',
    'メタリック５連ブレス',
    'ブローチ',
    '扇子',
    '暖簾',
    'のれん',
    '布',
    'ライト',
    'ウッドフロー',
    'ベンチ',
    'クリーパートレリス',
    'キャスケット',
    'けずり箸',
    'ミニチャーム',
    'ブレス',
    'テーブル',
    'クッション',
    'ジャケット',
    'タオル',
    'コサージュ',
    'がま口',
    'がまぐち',
    'スカーフ',
    'ティーコーゼ',
    'ブリキ鏡',
    'ファイルボックス',
    'つば広ラフィア帽',
    'サンバイザー',
    'ドライプラント',
    'イヤーマフ',
    'パンツ',
    'プルクロス',
    'パトロンデナバホドレス',
    'ヒンジ',
    'ｇｌａｓｓ',
    'ボックス',
    'チェックドレス',
    'ブックシェルフ',
    'ガーランド',
    'ノースリーブ',
    'ショール',
    'ディフューザー',
    'ｓｏｃｋｓ',
    'ｉｐｈｏｎｅ６／６ｓ カ  バー',
    'ｉｐｈｏｎｅ６／６Ｓ カ  バー',
    'ウォールラック',
    'クロシェ',
    'フラワーポット',
    'ポット',
    'バードハウス',
    'シューズ',
    'クロッシュバード',
    'ウォールステップラック',
    'ハンギング',
    'シャーレ',
    'グラス',
    'コンプロバールシャツ',
    'チョーカー',
    'プレーン',
    'フラットショルダー',
    'カード ラック',
    'カレンダー',
    'カバーウォーマー',
    'ホウキ',
    'シャツ',
    'ブランケット',
    'カバードｂｏｘ',
    'サンダル',
    '衿付きポンチョ',
    'カチューシャ',
    'アロマミスト',
    'ダストパン',
    'フリークロス',
    'コンフォートカバー',
    'オブジェ',
    'ラウンドスツール',
    'ティペット',
    'ティペッ ト',
    'ティペッ  ト',
    'カトラリーレスト',
    'ｍａｔ',
    'ティーセット',
    'ｔｅａセット',
    'プルオーバー',
    'プルオー  バー',
    'ボウル',
    'タペストリー',
    'ボトル',
    'チャーム',
    'イヤーカフ',
    'カチューム',
    'ストッカー',
    'ハーフスリーブ',
    'ハンドウォーマー',
    'トート',
    'ハンガーカバー',
    'コーディガン',
    'リース',
    'スプーン',
    'ネックウォーマー',
    'ロングベスト',
    'くつべら',
    'フレンチスリーブ',
    'ピン',
    'ｂｏｘ',
    'ワイヤー',
    'ベスト',
    'ヤツィドレス',
    'パッチドレス',
    'シェルフボード',
    'タンブラー',
    '防寒',
    'インセンス',
    'サイフ',
    'ダブルリム',
    'ベレー',
    'キャミソ',
    'メモカバー',
    'ペーパーハンガー',
    'エルソールドレス',
    'ハンガー',
    'ワックスコード',
    '電球コード',
    'ディナーナイフ',
    'ｆｕｒイヤーバンド',
    'キッチンセット',
    '木版',
    'ふちどり帽子',
    'ガウン',
    'アクセサリー',
    '巾着',
    '香立',
    'ｆｒａｍｅ',
    'ワゴン',
    'ローラー',
    'ホウロウ',
    'ホ−ロー',
    'つば広帽',
    '八角箸',
    'フラワー',
    'ローズマリー',
    'オリーブ',
    'タイツ',
    'サスペンダー',
    'アルファベット',
    'アンクレット',
    'バンダナ',
    'コート',
    'イヤーバンド',
    'ポンチョ',
    'リュック',
    'ランプ',
    'ドール',
    'ビーカー',
    'サロペット',
    'ペンダント',
    'スツール',
    'ボタン',
    'クレセントコーム',
    'ヒモ',
    'コロフドレス',
    'シェルフ',
    'トレイ',
    'ポシェット',
    'オーナメンタル',
    'キーカバー',
    'バスルームラック',
    'ガーゼ',
    'ラウンドハンドル',
    'ラック',
    'トピアリー',
    'スクープ',
    'ルエダドレス',
    'レードル',
    'ボード',
    'プランニングシザー',
    'プランター',
    'ディスプレイ',
    '更紗',
    'ロザリー',
    'クレッセントコーム',
    'フラスコ',
    'ネイル',
    'アンブレラグリッパー',
    'ドローアップチェスト',
    'レッグウ  ォーマー',
    'ショルダー',
    'レスト',
    'パントーンバンドル',
    'クロック',
    'マグ',
    'サラッペ',
    'ルーテ',
    'ポストカード',
    'オイル',
    'グリップ ハンドル',
    'メモパッド カバー',
    'ｐｏｔｔｅｒｙ',
    'パース',
    'シザー',
    'ブロック',
    '福袋',
    'ラッチＫＥＹ',
    'ラッチｋｅｙ',
    'ジョーロ',
    'コーム',
    'カード',
    'ラベンダーカモミール',
    'トップス',
    'トルソー',
    'ジレ',
    'ｖネック',
    'ボール',
    'クリスマス',
    '刺繍ドレ',
    'カゴ',
    'キッチンパット',
    'ルームブーツ',
    'ワンピー ス',
    'KNOB',
    'ｋｎｏｂ',
    'スタンプ',
    'ケープ',
    'レッグ',
    'フィンガーパペット',
    'サボ',
    'バインダー',
    'ホーロー',
    'リップバーム',
    'ハンチング',
    'スタッフド',
    'ベース',
    'ブーツウォーマー',
    'コンテナ',
    'ベビーパウダー',
    'ソーイングキット',
    'ドローアップチェスト',
    'ハニーコム',
    'コンパクトチェスト',
    'ブックマーク',
    'マドラー',
    'ウォール ポケット',
    'ロッカーチェスト',
    'コーヒードリッパー',
    'パイプ',
    'フルーツ',
    'ステッカー',
    'ピロー',
    'バード',
    'ベッドスプレット',
    'コンソール',
    'ウォールキャビネット',
    'ドローアップチェスト',
    'ドローアップ チェスト',
    'ストライプ',
    'ツリー',
    'アムレット',
    'ワンピー  ス',
    'ＢＡＧ',
    'パーカー',
    'ジャム',
    'ＨＡＴ',
    'レギパン',
    'リブＣＡＰ',
    'パウダー',
    'ＣＡＰ',
    '耳宛ＣＡＰ',
    'ＢＯＸ',
    'ＤＲＡＷＥＲ',
    'ＭＡＴ',
    'ＧＬＡＳＳ',
    'ＰＯＴＴＥＲＹ',
    'ドリンク',
    'ホ－ロー',
    'バタークリーム',
    'パルシュドレス',
    'ラフィア帽',
    'シューホーン',
    'ＦＲＡＭＥ',
    'ターナー',
    'ＫＮＯＢ',
    'プラントカバー',
    'スパチュラ',
    '灰皿',
    'オールインワン',
    'ジンジャーピーチ',
    'チーズ',
    'ストロベリー',
    'スワッグ',
    'インナードレス',
    'キャリーゴザ',
    'バケット',
    'Peg',
    'ペグ',
    'バターナイフ',
    'リャマ人形',
    'モチーフゴム',
    'クロス',
    'フーディ',
    'ジョーゴ',
    '子供長靴',
    '重箱',
    'ペーストナイフ',
    'むぎわら帽',
    'ブロウガラス',
    'ココナッツライム',
    'ブーケ',
    'キャビネット',
    '壁掛け',
    'Ｖネック',
    'ブーツ',
    'カギ',
    'カットソー',
    'ＣＡＮＤＬＥ',
    '糸',
    'ティピティピドレス',
    'カーデ',
    'チェーン',
    'バタービーター',
    'ベビーティアーズ',
    'メタルバッジ',
    'ケージ',
    'アイボルト',
    'ガイコツ',
    'ポ－チ',
    'ＳＯＣＫＳ',
    '根竹ハンドル',
    'アートタイル',
    'カバー',
    'ファイル',
    'ｉｐｈｏｎｅ６／６Ｓケー  ス',
    'ブックエンド',
    'パーツ',
    'ハウス',
    'ドイリー',
    'チャーチ',
    'ベチバー',
    'ロングＳＫ',
    'スノーフレーク',
    'ＪＡＲ',
    'キッズキャミ',
    'ＢＥＡＲ',
    'サンタ',
    'エストレジャス',
    'コンフィチュール',
    'クレイジーマンパ',
    'ＳＫ',
    '絵皿',
    'オレガノ',
    'バジル',
    'イカット',
    'スプレッド',
    'テスター',
    'セージ',
    'ユーカリ',
    'ハニーデュメロン',
    'ラベンダー',
    'プミラ',
    'サークルハイゲージ',
    'カラビナ',
    'シャカシャカ',
    'ＣＵＰ',
    'タブ',
    'レインディア',
    'エレサー',
    'ケット',
    'メンズ',
    'レディース'
]
product_names = [w.replace('\u3000', ' ') for w in category_df['product_name'].tolist()]
print("There are {} product names".format(len(product_names)))
unique_product_names = list(set(product_names))
print("There are {} unique product names".format(len(unique_product_names)))
removing_indexes = []
for index, c in enumerate(unique_product_names.copy()):
    for s in query_keywords:
        if s in c:
            removing_indexes.append(index)

for index in sorted(list(set(removing_indexes)), reverse=True):
    del unique_product_names[index]            

print("{} of products still remain unassigned with subcategory".format(len(unique_product_names)))
unique_product_names

There are 21863 product names
There are 11575 unique product names
203 of products still remain unassigned with subcategory


['１６Ｓ＆Ｓ Ｒｅｃｒｅａｔｉｏｎａｌ  Ｖｅｈｉｃｌｅ Ｎｏ．１',
 'ＮＡＰキルト ＴＷＩＬＬリネン ティント  ',
 'ＲＥＳＴＦＯＬＫ Ｎｏ．０９  ',
 'ＫＡＮＴＨＡ Ｂ・Ｓ  ',
 'グァテマラ Ｔ．Ｃ レインボー  ',
 'ｃｈｅｅｒ ２０１４ Ｓ＆Ｓ  ',
 'その他 部門５  ',
 '１４Ａ＆Ｗ Ｄｅｌｉｇｈｔｆｕｌ キッズ  ',
 'ツイストウール ＳＨＥＥＰ（Ｓ）  ',
 '代引き手数料  １万未満  ',
 'ＮＡＰキルト フルブロウ エンブロイダリー  ',
 'ＲＥＳＴＦＯＬＫ ＮＯ．７  ',
 'ソリッドファブリック ＤＡＨＬＩＡ  ',
 '特別値引 東京営業所  ',
 '２８１１２３９ＢＬ ＢＣ コモ  ',
 'ＲＥＳＴＦＯＬＫ ＮＯ．３  ',
 'プエブロバッファロー ロングＯＰ ＣＲＭ  ',
 '代引き手数料 ３０万未満  ',
 'ナチュラルウール ＨＯＲＳＥ（Ｌ）  ',
 '１７Ｓ＆Ｓ Ｃｏｍｉｄａ Ｃｏｒｒｉｄａ  ',
 'ＲＥＳＴＦＯＬＫ ＮＯ．１  ',
 'フラッフィーウール ＬＩＯＮ（Ｍ）  ',
 '１５Ｓ＆Ｓ Ｒｅｃｒｅａｔｉｏｎａｌ  Ｖｅｈｉｃｌｅ Ｎｏ．１',
 '２８１１２３９ＰＫ ＢＣ コモ  ',
 'カラーＪＱパッチデニムＰＴ Ｍ→Ｌ ＢＬ  ',
 'ブレイデッド バンクアン  ',
 'ウォッシュレザー コード  ',
 'Ｄ・Ｃ フル ブルーム  ',
 'オーガニックコットン キルトステッチ ＮＡＰ  ',
 '２８１１２３９ＭＴ ＢＣ コモ  ',
 'ＮＡＰキルト デインティ ローズ  ',
 '１４Ｓ＆Ｓ Ｄｅｌｉｇｈｔｆｕｌ 雑貨  ',
 'ＮＡＰキルト フラワリーローズ  ',
 '２８１１２３５ＭＡ ＢＣ タイダイイウユニ  ',
 'Ｔ／Ｃバルメリボーダー  ',
 'Ｄ・Ｃ フリスキーエンブロイダリー  ',
 '１７Ｓ＆Ｓ Ｒｅｃｒｅａｔｉｏｎａｌ  Ｖｅｈｉｃｌｅ Ｎｏ．１',
 '１７Ａ＆Ｗ Ｃｏｍｉｄａ Ｃｏｒｒｉｄａ  ',
 'ガラスウェイト マッシュルーム  ',
 '特別値引 外販営業  ',
 'ＭＲ４９７  ',
 '１６Ａ＆Ｗ Ｃｏｍｉｄａ Ｃｏｒｒｉｄａ  ',
 'ＭＲ５２７  ',
 'ｃｈｅｅｒ ２０１８ 

In [15]:
group_dict = {
    '財布': ['がまぐち','がま口','ウォレット','サイフ'],
    '帽子': ['帽','hat','キャップ','cap','ハット','hat'],
    'ドレス': ['ドレ','dress'],
    '箸' : ['箸','おはし'],
    'サンダル': ['サンダル'],
    'グリッパー': ['グリッパー'],
    'タイル': ['タイル'],
    'ウォーマー': ['ウォーマー'],
    '耳あて': ['イヤー'],
    'プレート': ['プレート'],
    'フロー': ['フロー'],
    '壁飾り': ['ウォール','壁掛け','壁飾り'],
    'バッグ': ['バッグ','bag','カゴ','ショルダー','リュック'],
    'スカート': ['sk','スカート'],
    'レスト': ['レスト'],
    'ウォーマー': ['ウォーマー'],
    'ボックス': ['box','ボックス','箱'],
    'カーディガン': ['カーデ'],
    'ラック': ['ラック','rack'],
    'ポット': ['ポット'],
    'スタンド':['スタンド'],
    'キャミソール': ['キャミ'],
    'キッチン用品': ['キッチン'],
    'クッション':['クッション'],
    'カバー': ['カバー'],
    'ランプ': ['ランプ'],
    'シャツ': ['シャツ'],
    'クレセントコーム': ['クレセントコーム','クレッセントコーム'],
    'グラス': ['グラス'],
    'ハンドル': ['ハンドル'],
    'ケース': ['ケース'],
    'ショール': ['ショール'],
    'サンタクロース': ['サンタ'],
    'スプーン':['スプーン'],
    'シューズ': ['シューズ','靴'],
    'ブーツ': ['ブーツ'],
    'ショート': ['ショート'],
    'パンツ': ['パンツ','ズボン','レギンス'],
    'スカーフ': ['スカーフ'],
    'ピン': ['ピン'],
    'スツール': ['スツール'],
    'ステッカー':['ステッカー'],
    'ストラップ': ['ストラップ','ストライプ'],
    'ソープディッシュ': ['ソープディッシュ','ソープデッシュ'],
    'テーブル': ['テーブル'],
    'クロス': ['クロス'],
    'ケージ': ['ケージ'],
    'ナイフ': ['ナイフ'],
    'トレー': ['トレー','トレイ','トレー'],
    'ピン': ['ピン'],
    'ノブ': ['knob','ノブ'],
    'ドロワー': ['ドロワー','drawer','ドロワ'],
    'ハンギング':['ハンギング'],
    'スリーブ': ['スリーブ'],
    'クリーム': ['クリーム'],
    'パペット': ['パペット','人形','ドール'],
    'ボウル': ['ボウル'],
    'エプロン': ['エプロン'],
    'ハサミ': ['シザー','ハサミ','scissor'],
    'クリップ': ['クリップ'],
    'ゴム': ['ゴム'],
    'ヘアーバンド': ['ヘアーバンド','ヘアバンド'],
    'ハンガー': ['ハンガー'],
    'カード': ['カード'],
    'パウダー': ['パウダー'],
    'ホルダー': ['ホルダー'],
    'マット': ['マット','mat'],
    '鏡': ['ミラー'],
    'チャーム': ['チャーム'],
    'ブレス': ['ブレス'],
    'バッジ': ['バッジ'],
    'ベスト': ['ベスト'],
    'ローブ': ['ローブ'],
    'グルーブ': ['手袋','グローブ'],
    '香立': ['アロマ','香立','インセンス','incent','インセント'],
    '時計': ['時計','クロック','腕時計','ウォッチ'],
    'pottery': ['皿','cup','カップ','コップ'],
    'ポンチョ': ['ポンチョ'],
    'コード': ['コード','cord'],
    'フレーム': ['frame','フレーム'],
    '携帯用品':['iphone'],
    'グラス': ['glass','グラス'],
    '靴下': ['靴下','socks','ソックス']
}


def get_groups(names,group_dict):
    groups = []
    for name in names:
        group_holder = []

        for key in group_dict:
            for query in group_dict[key]:
                if query in name:
                    group_holder.append(key)
                    break

        if group_holder:
            groups.append(",".join(group_holder))
        else:
            groups.append(name)
    return groups

query_keywords = sorted(query_keywords)
names = [ unicodedata.normalize('NFKC', name) for name in [ k.lower().replace(" ","") for k in query_keywords.copy()]]
groups = get_groups(names,group_dict)

dictionary = { 'query_keywords': query_keywords, 'name': names, 'group': groups }
new_df = pd.DataFrame(dictionary)
new_df


,group,name,query_keywords
0,ノブ,knob,KNOB
1,peg,peg,Peg
2,うちわ,うちわ,うちわ
3,財布,がまぐち,がまぐち
4,財布,がま口,がま口
5,くつべら,くつべら,くつべら
6,箸,けずり箸,けずり箸
7,帽子,つば広ラフィア帽,つば広ラフィア帽
8,帽子,つば広帽,つば広帽
9,のれん,のれん,のれん


### Export dictionaries as csv files

In [252]:
new_df.to_csv('processed_data/subcategory.csv', header=True, mode='w')